prompt:
Transfer learning on pretrained VGG 16 (see Lecture 17) to execute on colab

step 1: build vgg 16 pretrained model (tf.keras), following crisp-dm steps

step 2: modify the code to use medical-mask github for training (modify 2 class)

step 3: allow program to input image http , then classify which class it

安裝並導入必要的庫

In [1]:
# Install necessary packages (if needed)
!pip install tensorflow numpy matplotlib

# Import required libraries
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
import numpy as np
import matplotlib.pyplot as plt


加載VGG16預訓練模型

In [2]:
# Load the VGG16 model with pre-trained ImageNet weights, excluding the top layers
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the base model to prevent training on it
base_model.trainable = False

# Build the new model on top of the pre-trained VGG16
model = base_model.output
model = Flatten()(model)
model = Dense(256, activation='relu')(model)
model = Dropout(0.5)(model)
model = Dense(2, activation='softmax')(model)  # We have 2 classes: with_mask, without_mask

# Compile the model
model = Model(inputs=base_model.input, outputs=model)
# Compile the model with the correct learning rate argument
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])


# Summary of the model
model.summary()


58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)             │ (None, 224, 224, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block1_conv1 (Conv2D)                │ (None, 224, 224, 64)        │           1,792 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block1_conv2 (Conv2D)                │ (None, 224, 224, 64)        │          36,928 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block1_pool (MaxPooling2D)           │ (None, 112, 112, 64)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block2_conv1 (Conv2D)                │ (None, 112, 112, 128)       │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block2_conv2 (Conv2D)                │ (None, 112, 112, 128)       │         147,584 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block2_pool (MaxPooling2D)           │ (None, 56, 56, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_conv1 (Conv2D)                │ (None, 56, 56, 256)         │         295,168 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_conv2 (Conv2D)                │ (None, 56, 56, 256)         │         590,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_conv3 (Conv2D)                │ (None, 56, 56, 256)         │         590,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_pool (MaxPooling2D)           │ (None, 28, 28, 256)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_conv1 (Conv2D)                │ (None, 28, 28, 512)         │       1,180,160 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_conv2 (Conv2D)                │ (None, 28, 28, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_conv3 (Conv2D)                │ (None, 28, 28, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_pool (MaxPooling2D)           │ (None, 14, 14, 512)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_conv1 (Conv2D)                │ (None, 14, 14, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_conv2 (Conv2D)                │ (None, 14, 14, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_conv3 (Conv2D)                │ (None, 14, 14, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_pool (MaxPooling2D)           │ (None, 7, 7, 512)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 25088)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 256)                 │       6,422,784 │
├──────────────────────────────────────┼─────────────────────────────┼──────────────

 Total params: 21,137,986 (80.64 MB)

 Trainable params: 6,423,298 (24.50 MB)

 Non-trainable params: 14,714,688 (56.13 MB)

在VGG16模型上添加自定義分類器

In [3]:
!git clone https://github.com/chandrikadeb7/Face-Mask-Detection.git


Cloning into 'Face-Mask-Detection'...
remote: Enumerating objects: 4590, done.
remote: Total 4590 (delta 0), reused 0 (delta 0), pack-reused 4590 (from 1)
Receiving objects: 100% (4590/4590), 186.72 MiB | 27.36 MiB/s, done.
Resolving deltas: 100% (271/271), done.
Updating files: 100% (4155/4155), done.


In [4]:
import os
import shutil
from sklearn.model_selection import train_test_split

# Define the dataset path
base_dir = '/content/Face-Mask-Detection/dataset'
categories = ['with_mask', 'without_mask']

# Create 'train' and 'validation' directories
train_dir = os.path.join(base_dir, 'train')
val_dir = os.path.join(base_dir, 'validation')
os.makedirs(train_dir, exist_ok=True)
os.makedirs(val_dir, exist_ok=True)

# Loop through categories and split images
for category in categories:
    category_path = os.path.join(base_dir, category)
    images = os.listdir(category_path)
    train_images, val_images = train_test_split(images, test_size=0.2, random_state=42)

    # Create subdirectories for each category
    os.makedirs(os.path.join(train_dir, category), exist_ok=True)
    os.makedirs(os.path.join(val_dir, category), exist_ok=True)

    # Move the images to the appropriate directories
    for img in train_images:
        shutil.move(os.path.join(category_path, img), os.path.join(train_dir, category, img))
    for img in val_images:
        shutil.move(os.path.join(category_path, img), os.path.join(val_dir, category, img))


修改代碼以使用醫療口罩資料集進行訓練

In [5]:
train_datagen = ImageDataGenerator(
    rescale=1./255,  # Rescale pixel values to [0,1]
    rotation_range=30,  # Random rotations
    width_shift_range=0.2,  # Random shifts
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'  # Fill missing pixels after transformations
)

test_datagen = ImageDataGenerator(rescale=1./255)

# Set up data generators for training and validation
train_generator = train_datagen.flow_from_directory(
    '/content/Face-Mask-Detection/dataset/train',  # Training data directory
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'  # For 2 classes: with_mask and without_mask
)

validation_generator = test_datagen.flow_from_directory(
    '/content/Face-Mask-Detection/dataset/validation',  # Validation data directory
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)


Found 3273 images belonging to 2 classes.
Found 819 images belonging to 2 classes.


In [6]:
!pip install Pillow
import requests
from PIL import Image
from io import BytesIO


In [7]:
def preprocess_image(url):
    # Download image from URL
    response = requests.get(url)
    img = Image.open(BytesIO(response.content))

    # Resize image to 224x224 (VGG16 input size)
    img = img.resize((224, 224))

    # Convert image to numpy array and normalize it
    img_array = np.array(img) / 255.0

    # Ensure the shape is (1, 224, 224, 3)
    img_array = np.expand_dims(img_array, axis=0)

    return img_array


準備資料集進行訓練

In [8]:
def classify_image(model, image_url):
    img_array = preprocess_image(image_url)

    # Make prediction
    predictions = model.predict(img_array)
    class_index = np.argmax(predictions, axis=1)[0]

    # Map the class index to the class label
    class_labels = ['with_mask', 'without_mask']
    predicted_class = class_labels[class_index]

    return predicted_class


訓練模型

In [9]:
image_url = 'https://na.cx/i/eqzQJYw.jpg'  # Replace with an actual URL
result = classify_image(model, image_url)
print(f"The image belongs to class: {result}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 5s 5s/step
The image belongs to class: without_mask


允許程式通過HTTP URL輸入圖像並進行分類

In [10]:
history = model.fit(
    train_generator,
    epochs=10,
    validation_data=validation_generator
)


Epoch 1/10


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


 51/103 ━━━━━━━━━━━━━━━━━━━━ 23s 448ms/step - accuracy: 0.7147 - loss: 0.5995

/usr/local/lib/python3.10/dist-packages/PIL/Image.py:1054: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


103/103 ━━━━━━━━━━━━━━━━━━━━ 85s 607ms/step - accuracy: 0.7778 - loss: 0.4809 - val_accuracy: 0.9719 - val_loss: 0.0989
Epoch 2/10
103/103 ━━━━━━━━━━━━━━━━━━━━ 52s 472ms/step - accuracy: 0.9343 - loss: 0.1768 - val_accuracy: 0.9792 - val_loss: 0.0619
Epoch 3/10
103/103 ━━━━━━━━━━━━━━━━━━━━ 54s 506ms/step - accuracy: 0.9504 - loss: 0.1354 - val_accuracy: 0.9841 - val_loss: 0.0499
Epoch 4/10
103/103 ━━━━━━━━━━━━━━━━━━━━ 51s 472ms/step - accuracy: 0.9574 - loss: 0.1207 - val_accuracy: 0.9805 - val_loss: 0.0485
Epoch 5/10
103/103 ━━━━━━━━━━━━━━━━━━━━ 81s 453ms/step - accuracy: 0.9534 - loss: 0.1237 - val_accuracy: 0.9878 - val_loss: 0.0472
Epoch 6/10
103/103 ━━━━━━━━━━━━━━━━━━━━ 49s 454ms/step - accuracy: 0.9525 - loss: 0.1194 - val_accuracy: 0.9841 - val_loss: 0.0418
Epoch 7/10
103/103 ━━━━━━━━━━━━━━━━━━━━ 83s 463ms/step - accuracy: 0.9561 - loss: 0.1108 - val_accuracy: 0.9866 - val_loss: 0.0401
Epoch 8/10
103/103 ━━━━━━━━━━━━━━━━━━━━ 50s 449ms/step - accuracy: 0.9631 - loss: 0.0978 - val